# Setup

In [ ]:
VERSION = 'pspfgp-48-nn-pretrain'
DEV = False
DATA = 'ALL'  # COMP, ALL
N_BAGS = 1
N_FOLDS = 5
N_SEEDS = 1
D_MODEL = 24
BATCH_SIZE = 128
EXPIT = True
SEED = 0
GPU = 0
N_THREADS = 8
VERBOSE = True
LEVEL_GROUPS = ['0-4', '5-12', '13-22']
PREV_LEVEL_GROUPS = False
LENGTHS = {'0-4': 600, '5-12': 1400, '13-22': 2000}

In [ ]:
DISCRETE_FEATURES = ['room_fqid', 'event_name_name', 'text', 'fqid']
CONTINUOUS_FEATURES = ['duration']
FEATURES = DISCRETE_FEATURES + CONTINUOUS_FEATURES

In [ ]:
from IPython.display import clear_output

In [ ]:
from IPython.display import display, HTML
display(HTML('<style>td{white-space: nowrap !important;}</style>'))

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import pickle
from tqdm.notebook import tqdm
import tensorflow as tf
import time

In [ ]:
if EXPIT:
    from scipy.special import expit, logit

In [ ]:
tf.get_logger().setLevel('WARNING')

In [ ]:
import random

os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'  # TF will not use all memory

In [ ]:
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 250)

# Utils

In [ ]:
def get_questions(level_group):
    return ([1, 2, 3] if level_group == '0-4' 
            else [4, 5, 6, 7, 8, 9, 10, 11, 12, 13] if level_group == '5-12' 
            else [14, 15, 16, 17, 18])

In [ ]:
def optimize_threshold(preds, labels, step_size=0.001):
    thresholds = []
    best_score = 0
    best_threshold = 0
    for threshold in np.arange(0, 1, step_size):
        binarized_preds = (preds.melt().drop('variable', axis=1).values > threshold).astype('int')
        score = fast_f1_score(labels.melt().drop('variable', axis=1).values, binarized_preds)
        thresholds.append(threshold)
        if score > best_score:
            best_score = score
            best_threshold = np.round(threshold, 3)
    return best_threshold

In [ ]:
def optimize_thresholds(preds, labels, threshold):
    thrs = []
    binarized_preds = preds.copy()
    binarized_preds = 1 * (binarized_preds >= threshold)
    binarized_preds
    for q in range(1, 18 + 1):
        best_score = 0
        for thr in np.arange(0.5, 0.8, 0.005):
            oof_preds = binarized_preds.copy()
            oof_preds[f'q{q}'] = 1 * (preds[[f'q{q}']] >= thr)
            score = score_questions(oof_preds, labels, range(1, 18 + 1), thr=thr)['overall']
            if score > best_score:
                best_score = score
                best_thr = thr
        thrs.append(best_thr)
    return thrs

In [ ]:
def fast_f1_score(labels, preds):
    all_positives = (preds + labels == 2).mean()
    all_negatives = (preds + labels == 0).mean()
    score = 1 - (1 - all_negatives - all_positives) / (1 - (all_negatives - all_positives) ** 2)
    return score

In [ ]:
def score_questions(preds, labels, questions, thr):
    thresholds = thr if type(thr) == list else [thr] * 18
    scores = dict()
    preds_binarized = preds.copy()
    for question in questions:
        preds_binarized[f'q{question}'] = (preds[f'q{question}'].values > thresholds[question - 1]).astype('int')
        score = fast_f1_score(labels[f'q{question}'].values, preds_binarized[f'q{question}'].values)
        scores[f'q{question}'] = np.round(score, decimals=5)
    if len(questions) > 1:
        score = fast_f1_score(labels.melt().drop('variable', axis=1).values, 
                              preds_binarized.melt().drop('variable', axis=1).values)
        scores['overall'] = np.round(score, decimals=5)
    return scores

In [ ]:
def tokenize(df, feats):
    df_out = df.copy()
    tokenizer_map = {}
    for f in feats:
        factorized = pd.factorize(df[f])
        df_out[f] = factorized[0] + 1
        tokenizer_map[f] = {
            'encode': ['<PAD>'] + list(factorized[1]),
            'decode': {(i + 1): el for i, el in enumerate(factorized[1])}
        }
    return df_out, tokenizer_map

In [ ]:
def compute_score(y_pred, y_true, threshold=None):
    cols = [f'q{q}' for q in range(1, 18 + 1)]
    df = pd.DataFrame(index=y_true.index)
    df.loc[:, cols] = np.hstack([y_pred[level_group] for level_group in LEVEL_GROUPS])
    if threshold is None:
        threshold = optimize_threshold(df, y_true)
    scores = score_questions(y_true, 1 * (df > threshold), range(1, 18 + 1), threshold)
    scores['thr'] = threshold
    return scores, df, threshold

In [ ]:
def build_sequence(df, n_features, length):
    return (np.vstack([df.values, np.zeros((length - len(df), n_features))]) 
            if len(df) < length else df[:length].values)

In [ ]:
def extract_targets(dataset):
    targets = []
    for level_group in LEVEL_GROUPS:
        t = [b[1][level_group] for b in list(dataset)]
        targets.append(np.vstack(t))
    targets = np.hstack(targets)
    return targets

# Data

In [ ]:
METADATA = pickle.load(open(f'../final/METADATA.pkl', 'rb'))
train_datasets = pickle.load(open(f'../data/processed/train_datasets.pkl', 'rb'))
Y = pickle.load(open(f'../data/processed/Y.pkl', 'rb'))

# Model

## Architecture

In [ ]:
class ConvBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, dropout_rate):
        super(ConvBlock, self).__init__()
        self.conv1d = tf.keras.layers.Conv1D(d_model, kernel_size=5, padding='same', activation='gelu')
        self.layer_norm = tf.keras.layers.LayerNormalization()
        self.dropout = tf.keras.layers.Dropout(rate=dropout_rate)
        
    def call(self, inputs):
        x = self.conv1d(inputs)
        x = x + inputs
        x = self.layer_norm(x)
        outputs = self.dropout(x)
        return outputs

In [ ]:
class TimeEmbedding(tf.keras.layers.Layer):
    def __init__(self, n_blocks, d_model, dropout_rate):
        super(TimeEmbedding, self).__init__()
        self.conv_blocks = [ConvBlock(d_model, dropout_rate=dropout_rate) for _ in range(n_blocks)]
        
    def call(self, inputs):
        x = tf.expand_dims(inputs, axis=-1)
        for conv_block in self.conv_blocks:
            x = conv_block(x)        
        return x

In [ ]:
class ConvNet(tf.keras.Model):
    def __init__(self, input_dims, n_outputs, d_model, n_blocks=4, name=None):
        super(ConvNet, self).__init__(name=name)
        self.input_dims = input_dims
        self.n_outputs = n_outputs
        self.d_model = d_model
        self.n_blocks = n_blocks
        self.event_embedding = tf.keras.layers.Embedding(input_dims['event_name_name'], d_model, mask_zero=True)
        self.room_embedding = tf.keras.layers.Embedding(input_dims['room_fqid'], d_model, mask_zero=True)
        self.text_embedding = tf.keras.layers.Embedding(input_dims['text'], d_model, mask_zero=True)
        self.fqid_embedding = tf.keras.layers.Embedding(input_dims['fqid'], d_model, mask_zero=True)
        self.duration_embedding = TimeEmbedding(n_blocks=n_blocks, d_model=d_model, dropout_rate=0.2)
        self.gap = tf.keras.layers.GlobalAveragePooling1D()
        
    def call(self, inputs):
        event = self.event_embedding(inputs['event_name_name'])
        room = self.room_embedding(inputs['room_fqid'])
        text = self.text_embedding(inputs['text'])
        fqid = self.fqid_embedding(inputs['fqid'])
        duration = self.duration_embedding(inputs['duration'])
        x = duration * (event + room + text + fqid)
        outputs = self.gap(x)
        return outputs

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'input_dims': self.input_dims,
            'n_outputs': self.n_outputs,
            'd_model': self.d_model,
            'n_blocks': self.n_blocks,
            'name': self._name,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
class SimpleHead(tf.keras.Model):
    def __init__(self, n_units, n_outputs, name=None, dropout_rate=0.2):
        super(SimpleHead, self).__init__(name=name)
        self.ffs = [tf.keras.layers.Dense(units, activation='leaky_relu') for units in n_units]
        self.out = tf.keras.layers.Dense(n_outputs, activation='sigmoid')
        
    def call(self, inputs):
        x = inputs
        for i in range(len(self.ffs)):
            x = self.ffs[i](x)
        outputs = self.out(x)
        return outputs

In [ ]:
def build_model(convnets, heads, trainable=False):
    inputs = {}
    for level_group in LEVEL_GROUPS:
        inputs[level_group] = {}
        for feature in FEATURES:
            name = f'input_{feature}_{level_group.replace("-", "_")}'
            inputs[level_group][feature] = tf.keras.Input(shape=(LENGTHS[level_group]), name=name) 
    
    for level_group in LEVEL_GROUPS:
        convnets[level_group].trainable = trainable

    convnet_outputs = {level_group: convnets[level_group](inputs[level_group]) for level_group in LEVEL_GROUPS}

    outputs = {}
    outputs['0-4'] = heads['0-4'](convnet_outputs['0-4'])
    outputs['5-12'] = heads['5-12'](
        tf.keras.layers.Concatenate(name='concat_5_12')(
            [convnet_outputs['0-4'], convnet_outputs['5-12']]))
    outputs['13-22'] = heads['13-22'](
        tf.keras.layers.Concatenate(name='concat_13_22')(
            [convnet_outputs['0-4'], convnet_outputs['5-12'], convnet_outputs['13-22']]))

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='pspfgp_model')
    
    return model

In [ ]:
def get_models():
    models = {}
    custom_objects = {'ConvNet': ConvNet, 'SimpleHead': SimpleHead}
    with tf.keras.utils.custom_object_scope(custom_objects):
        for level_group in LEVEL_GROUPS:
            models[level_group] = {}
            for b in range(N_BAGS):
                for f in range(N_FOLDS):
                    prefix = f'../models/{VERSION}/'
                    suffix = f'_{level_group.replace("-", "_")}_b{b}f{f}.h5'
                    models[level_group][f'b{b}f{f}'] = {}
                    convnet = tf.keras.models.load_model(f'{prefix}convnet{suffix}', compile=False)
                    models[level_group][f'b{b}f{f}']['convnet'] = convnet
    return models

## Train

In [ ]:
class EvaluationCallback(tf.keras.callbacks.Callback):
    def __init__(self, val_dataset, val_session_ids, test_dataset=None, test_session_ids=None):
        self.val_dataset = val_dataset
        self.val_session_ids = val_session_ids
        self.test_dataset = test_dataset
        self.test_session_ids = test_session_ids
        self.val_preds = None
        self.journal = []

    def on_epoch_end(self, epoch, logs=None):
        info = {}
        info['logs'] = logs.copy()
        for k, v in info['logs'].items():
            info['logs'][k] = np.round(v, 5)
        preds = self.model.predict(self.val_dataset, verbose=False)
        targets = pd.DataFrame(index=self.val_session_ids, 
                               data=extract_targets(self.val_dataset), 
                               columns=[f'q{q}' for q in range(1, 18 + 1)])
        info['val'], self.val_preds, threshold = compute_score(preds, targets, 0.625)
        if self.test_dataset is not None:
            preds = self.model.predict(self.test_dataset, verbose=False)
            targets = pd.DataFrame(index=self.test_session_ids, 
                                   data=extract_targets(self.test_dataset), 
                                   columns=[f'q{q}' for q in range(1, 18 + 1)])
            info['test'], _, _ = compute_score(preds, targets, threshold)
        logs['f1_score'] = info['val']['overall']
        self.journal.append(info)

In [ ]:
class LogCallback(tf.keras.callbacks.Callback):
    def __init__(self, evaluation_callback):
        self.evaluation_callback = evaluation_callback
        self.journal = []
        self.t_0 = time.time()

    def on_epoch_end(self, epoch, logs=None):
        val_score = self.evaluation_callback.journal[-1]['val']['overall']
        log = {
            'epoch': epoch + 1,
            'time': int(time.time() - self.t_0),
            'lr': logs['lr'],
            'loss': logs['loss'],
            'val_loss': logs['val_loss'],
            'val_score': val_score,
            'val_0-4_loss': logs['val_0-4_loss'],
            'val_5-12_loss': logs['val_5-12_loss'],
            'val_13-22_loss': logs['val_13-22_loss'],
        }
        
        if 'test' in self.evaluation_callback.journal[-1]:
            test_score = self.evaluation_callback.journal[-1]['test']['overall']
            log['test_score'] = test_score
            
        self.journal.append(log)
        
        if epoch == 0:
            if 'test' in self.evaluation_callback.journal[-1]:
                print('{:<8}{:<11}{:<9}{:<10}{:<11}{:<9}{:<9}{:<9}{:<11}{:<11}'
                      .format('Epoch', 'Time', 'lr', 'Loss', 'Val loss', '0-4', '5-12', '13-22', 'Val', 'Test'))
            else:
                print('{:<8}{:<11}{:<9}{:<10}{:<11}{:<9}{:<9}{:<9}{:<11}'
                      .format('Epoch', 'Time', 'lr', 'Loss', 'Val loss', '0-4', '5-12', '13-22', 'Val'))
            
        best_loss = min([h['val_loss'] for h in self.journal])
        best_val_score = max([h['val_score'] for h in self.journal])
        if 'test' in self.evaluation_callback.journal[-1]:
            best_test_score = max([h['test_score'] for h in self.journal])
        info = self.journal[-1]
        hours = str(info['time'] // 3600).zfill(2)
        minutes = str(info['time'] // 60 % 60).zfill(2)
        seconds = str(info['time'] % 60).zfill(2)
        
        if 'test' in self.evaluation_callback.journal[-1]:
            print('{:<8}{:<11}{:<9}{:<10}{:<11}{:<9}{:<9}{:<9}{:<11}{:<11}'.format(
                f'{epoch + 1}',
                '{}:{}:{}'.format(hours, minutes, seconds),
                f"{round(info['lr'], 4):06.4f}",
                f"{round(info['loss'], 5):07.5f}",
                f"{round(info['val_loss'], 5):07.5f}" + ('-' if info['val_loss'] == best_loss else ''),
                f"{round(info['val_0-4_loss'], 4):06.4f}",
                f"{round(info['val_5-12_loss'], 4):06.4f}",
                f"{round(info['val_13-22_loss'], 4):06.4f}",
                f"{round(info['val_score'], 5):07.5f}" + ('+' if info['val_score'] == best_val_score else ''),
                f"{round(info['test_score'], 5):07.5f}" + ('+' if info['test_score'] == best_test_score else ''),
            ))
        else:
            print('{:<8}{:<11}{:<9}{:<10}{:<11}{:<9}{:<9}{:<9}{:<11}'.format(
                f'{epoch + 1}',
                '{}:{}:{}'.format(hours, minutes, seconds),
                f"{round(info['lr'], 4):06.4f}",
                f"{round(info['loss'], 5):07.5f}",
                f"{round(info['val_loss'], 5):07.5f}" + ('-' if info['val_loss'] == best_loss else ''),
                f"{round(info['val_0-4_loss'], 4):06.4f}",
                f"{round(info['val_5-12_loss'], 4):06.4f}",
                f"{round(info['val_13-22_loss'], 4):06.4f}",
                f"{round(info['val_score'], 5):07.5f}" + ('+' if info['val_score'] == best_val_score else ''),
            ))

In [ ]:
def plot_scores(logs):
    plt.figure(figsize=(8, 3))
    plt.plot([l['val']['overall'] for l in logs])
    legend = ['val']
    if DATA == 'COMP':
        plt.plot([l['test']['overall'] for l in logs])
    legend.append('test')
    plt.legend(legend)
    plt.show()

In [ ]:
def schedule_lr(epoch):
    if epoch < 20:
        return 1e-3
    elif epoch < 30:
        return 5e-4
    elif epoch < 40:
        return 2.5e-4
    else:
        return 1e-4

In [ ]:
class LearningRateSchedulerCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        self.best = 0
        self.n_steps_since_last_best = 0
        self.n_steps_of_best = 0

    def on_epoch_end(self, epoch, logs=None):
        if logs['f1_score'] > self.best:
            self.best = logs['f1_score']
            self.n_steps_since_last_best = 0
            self.n_steps_of_best += 1
        else:
            self.n_steps_since_last_best += 1
            self.n_steps_of_best = 0
        lr_backup = lr = self.model.optimizer.lr.read_value()
        lr = lr + 1e-4 if self.n_steps_of_best > 2 else lr - 1e-4 if self.n_steps_since_last_best > 2 else lr
        lr = 1e-3 if lr > 1e-3 else 1e-4 if lr < 1e-4 else lr
        logs['lr'] = np.round(lr_backup, 4)
        self.model.optimizer.lr.assign(lr)

In [ ]:
early_stopping_round_callback = tf.keras.callbacks.EarlyStopping(
    monitor='f1_score',
    min_delta=0,
    patience=20,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True,
)

In [ ]:
%%time

random.seed(SEED)
tf.random.set_seed(SEED)

models = get_models()
session_ids = METADATA[METADATA['fold'] > -1]['session_id'].drop_duplicates().tolist()
oof_preds = pd.DataFrame(index=session_ids, 
                         data=np.zeros((len(session_ids), 18)), 
                         columns=[f'q{q}' for q in range(1, 18 + 1)])
labels = Y[Y['session_id'].isin(session_ids)].reset_index(drop=True)
logs = []

for b in range(N_BAGS):
    for f in range(N_FOLDS):
        print(f'> Bag {b} Fold {f}' + "\n")
        
        convnets = {level_group: models[level_group][f'b{b}f{f}']['convnet'] for level_group in LEVEL_GROUPS}
        heads = {}
        for level_group in LEVEL_GROUPS:
            n_units = [512, 128, 64]
            n_outputs = len(get_questions(level_group))
            heads[level_group] = SimpleHead(n_units, n_outputs, level_group)
            
        model = build_model(convnets, heads)

        metadata = METADATA[METADATA['train'] & (METADATA['fold'] == f)]
        val_session_ids = metadata['session_id'].drop_duplicates().tolist()

        (x_train, y_train), (x_val, y_val) = train_datasets[f'f{f}']
        train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
        train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
        val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
        val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
        
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
        loss = tf.keras.losses.BinaryCrossentropy()
        model.compile(loss=loss, optimizer=optimizer)
        
        evaluation_callback = EvaluationCallback(val_dataset, val_session_ids, None, None)
        log_callback = LogCallback(evaluation_callback)
        lr_callback = LearningRateSchedulerCallback()
        callbacks = [evaluation_callback, lr_callback, log_callback, early_stopping_round_callback]
        history = model.fit(
            train_dataset, validation_data=val_dataset, epochs=100, verbose=False, callbacks=callbacks)
        
        for level_group in LEVEL_GROUPS:
            heads[level_group]._name = f'head_{level_group.replace("-", "_")}_b{b}f{f}'
            models[level_group][f'b{b}f{f}']['head'] = heads[level_group]
        
        logs.append(evaluation_callback.journal)
        
        plot_scores(logs[b * f + f])
        
        oof_preds.loc[val_session_ids] = evaluation_callback.val_preds.values
        
        print()

In [ ]:
oof_preds

In [ ]:
model.summary()

# Export

In [ ]:
VERSION = f'pspfgp-48-nn-simple-head-s{SEED}'

In [ ]:
! rm -rf ../models/"$VERSION"
! mkdir ../models/"$VERSION"

In [ ]:
# ls -lh ../models

In [ ]:
%%time

tf.get_logger().setLevel('ERROR')

for i, level_group in enumerate(LEVEL_GROUPS):
    for b in range(N_BAGS):
        for f in range(N_FOLDS):
            inputs = {}
            suffix = level_group.replace("-", "_")
            for feature in FEATURES:
                inputs[feature] = tf.keras.Input(shape=(LENGTHS[level_group]), name=f'input_{feature}_{suffix}') 
            outputs = models[level_group][f'b{b}f{f}']['convnet'](inputs)
            convnet = tf.keras.Model(inputs=inputs, outputs=outputs, name=f'convnet_{suffix}')
            convnet.save(f'../models/{VERSION}/convnet_{suffix}_b{b}f{f}.h5', include_optimizer=False)
            
            inputs = tf.keras.Input(shape=(24 * (i + 1)), name=f'input') 
            outputs = models[level_group][f'b{b}f{f}']['head'](inputs)
            head = tf.keras.Model(inputs=inputs, outputs=outputs, name=f'head_{suffix}')
            head.save(f'../models/{VERSION}/head_{suffix}_b{b}f{f}.h5', include_optimizer=False)

tf.get_logger().setLevel('WARNING')

In [ ]:
%%time
custom_objects = {'ConvNet': ConvNet}
for level_group in LEVEL_GROUPS:
    for f in range(N_FOLDS):
        file_name = f"convnet_{level_group.replace('-', '_')}_b0f{f}"
        path = f'../models/{VERSION}/{file_name}'
        print(path)
        with tf.keras.utils.custom_object_scope(custom_objects):
            convnet = tf.keras.models.load_model(f'{path}.h5')
        converter = tf.lite.TFLiteConverter.from_keras_model(convnet)
#         converter.optimizations = [tf.lite.Optimize.DEFAULT]
#         converter.target_spec.supported_types = [tf.float16]
        tflite_model = converter.convert()
        with open(f'{path}.tflite', 'wb') as f:
            f.write(tflite_model)

In [ ]:
%%time
custom_objects = {'SimpleHead': SimpleHead}
for level_group in LEVEL_GROUPS:
    for f in range(N_FOLDS):
        file_name = f"head_{level_group.replace('-', '_')}_b0f{f}"
        path = f'../models/{VERSION}/{file_name}'
        print(path)
        with tf.keras.utils.custom_object_scope(custom_objects):
            convnet = tf.keras.models.load_model(f'{path}.h5')
        converter = tf.lite.TFLiteConverter.from_keras_model(convnet)
#         converter.optimizations = [tf.lite.Optimize.DEFAULT]
#         converter.target_spec.supported_types = [tf.float16]
        tflite_model = converter.convert()
        with open(f'{path}.tflite', 'wb') as f:
            f.write(tflite_model)

In [ ]:
pickle.dump(oof_preds, open(f'../models/{VERSION}/oof_preds.pkl', 'wb'))

In [ ]:
ls ../models/"$VERSION"